In [36]:
file = "data/a_example.in"
# file = "data/b_should_be_easy.in"
# file = "data/c_no_hurry.in"
# file = "data/d_metropolis.in"

In [49]:

lines = []
with open(file, 'r') as f:
    lines = f.read().split(sep = '\n')
    print(lines[-1])

def validateLine(s: str) -> bool:
    if s == '':
        return False
    n = list(map(int, s.split(sep = ' ')))
    return abs(n[0] - n[2]) + abs(n[1] - n[3]) <= n[5] - n[4]



def distance(start, end):
    return abs(start[0] - end[0]) + abs(start[1] - end[1])


def in_range(begin, end, time):
    return distance(begin, end) <= time


def takeWhile(it: iter, p):
    """ Takes from the iterator as long as the predicate is holding true
    Args:
        it (iter): The iterator in question
        p (predicate): The Predicate deciding when to stop
        
    Returns:
        Iterator including all elements up to the point before the predicate said 'False'
    """
    for i in it:
        if p(i):
            yield i
        else:
            raise StopIteration


def dropWhile(it: iter, p):
    """ Drops from the iterator as long as the predicate is holding true
    Args:
        it (iter)dv: The iterator in qeustion
        p (predicate): The Predicate deciding when to start
        
    Returns:
        Iterator with all the elements starting from when the predicate said 'False'
    """
    b = False
    for i in it:
        if b or not p(i):
            b = True
            yield i


## validating if they are all valid
# for i, l in enumerate(lines):
#     if not validateLine(l):
#         print(i)
# list(map(validateLine, lines))


In [54]:
str.join?

In [79]:
class Ride:
    _id = 0
    _place_begin = None
    _place_end = None
    _time_begin = None
    _time_end = None
    
    def __init__(self, line: str, i: int):
        """ Initialize this route based on the given line.
        needs to contain six numbers.
        i is the
        """
        n = list(map(int, line.split(sep = ' ')))
        self._id = i
        self._place_begin = (n[0], n[1])
        self._place_end = (n[2], n[3])
        self._time_begin = n[4]
        self._time_end = n[5]
        
    def __repr__(self):
        return "From {} to {} between {} and {}".format(self._place_begin, self._place_end,
                                                        self._time_begin, self._time_end)

    def get_begin_time(self):
        return self._time_begin
    
    def get_time_duration(self):
        return self._time_end - self._time_begin

    def get_id(self):
        return self._id
    
    def get_begin_place(self):
        return self._place_begin
    
    def get_end_place(self):
        return self._place_end


class Car:
    _id = 0
    _location = [1, 1] # (x, y)
    _destination = None
    _ultimate = None
    _rides = []
    _world = None
    _state = 0
    # 0 : search for next ride
    # 1 : driving to next ride
    # 2 : driving to destination of ride
    
    def __init__(self, i: int, world: World):
        self._id = i
        self._world = world

    def make_step(self):
        
        if self._location == self._destination:
            if self._state == 1:
                self._destination = self._ultimate
            self._state = self._state + 1 % 3
        
        n_old = 0
        n = 50
        while self._state == 0:
            rides = self._world.get_next_rides(n)
            for r in rides[n_old:n]:
                if in_range(r.get_begin_place(), r.get_end_place(), r.get_time_duration()):
                    self._rides.append(r)
                    self._world.make_ride(r.get_id())
                    self._state = 1
                    self._destination = r.get_begin_place()
                    self._ultimate = r.get_end_place()
                    break
            n_old = n
            n += 20
        
        if self._state == 1 or self._state == 2:
            if self._location[0] < self._destination[0]:
                self._location[0] += 1
            elif self._location[0] > self._destination[0]:
                self._location[0] -= 1
            elif self._location[1] < self._destination[1]:
                self._location[1] += 1
            elif self._location[1] > self._destination[1]:
                self._location[1] -= 1

    def __str__(self):
        return str(self._id) + ' ' + ' '.join(map(lambda r: str(r.get_id()), self._rides))

    def __repr__(self):
        return str(self)

class World:
    _size = (1, 1) # (x, y)
    _num_cars = 1
    _num_rides = 1
    _bonus = 1
    _max_timesteps = 1
    _current_timestep = 0
    _rides = []
    _rides_sorted = []
    _cars = []
    
    _filename = None
    
    def __init__(self, filename):
        self._filename = filename[:-2] + 'out'
        lines = []
        with open(file, 'r') as f:
            lines = f.read().split(sep = '\n')[:-1]
        
        # assign directly readable values
        n = list(map(int, lines[0].split(sep = ' ')))
        self._size = (n[0], n[1])
        self._num_cars = n[2]
        self._num_rides = n[3]
        self._bonus = n[4]
        self._max_timesteps = n[5]

        # read the rides in
        for i, l in enumerate(lines[1:]):
            if validateLine(l):
                self._rides.append(Ride(l, i))
                
        # sort the rides chronologically
        self._rides_sorted = sorted(self._rides, key = lambda x: x.get_begin_time())
        
        for i in range(n[2]):
            self._cars.append(Car(i, self))
        

    def get_next_rides(self, n = 50):
        # return the next n rides, default 50
        return self._rides_sorted[:n]

    def make_ride(self, ride_id: int):
        self._rides_sorted[:ride_id + 1] = self._rides_sorted[:ride_id]

    def make_timestep(self):
        for c in self._cars:
            c.make_step()
        self._current_timestep += 1
        if self._current_timestep % 50 == 0:
            self._rides_sorted = list(dropWhile(lambda r: r.get_begin_time() > self._current_timestep))
    
    def finish(self):
        while self._current_timestep < self._max_timesteps:
            self.make_timestep()
        return self._cars



In [82]:
w = World("data/a_example.in")
c = w.finish()

In [83]:
c

[0 1 1 1 1, 1 1 1 1 1, 0 1 1 1 1, 1 1 1 1 1]